In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import re
from collections import Counter
import nltk
import string
from nltk.corpus import stopwords

In [ ]:
import os
print(os.listdir("C:/Users/91982/Desktop/NTCC"))

In [ ]:
df= pd.read_csv("C:/Users/91982/Desktop/NTCC/Reviews.csv")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
data1=pd.DataFrame(df.groupby('Score').size().sort_values(ascending=False).rename('No of Users').reset_index())
data1.head()

In [ ]:
data2 = df[['Score' , 'Text' , 'Summary' , 'UserId']]
finaldata=data2.dropna()
finaldata.head()

In [ ]:
finaldata['Summary'].fillna("Good", inplace = True)

In [ ]:
finaldata.isnull().sum()

In [ ]:
finaldata["Sentiment"] = finaldata["Score"].apply(lambda score: "positive" if score > 3 else "negative")
finaldata['Sentiment'] = finaldata['Sentiment'].map({'positive':1, 'negative':0})

In [ ]:
finaldata.columns

In [ ]:
Senti = finaldata[(finaldata['Sentiment']== 0) | (finaldata['Sentiment']==1)]
print(Senti)

In [ ]:
y = Senti['Sentiment']
x = Senti['Summary']
print(x)

In [ ]:
print(y)

In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
text = Senti['Text']
labels = Senti['Sentiment']

In [ ]:
def clean_document(doco):
    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)
    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')
    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])
    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    doco_clean = ([i for i in doco_clean if i not in stop])
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])

    return doco_clean

In [ ]:
clean = [clean_document(doc) for doc in text];
sentence = [' '.join(r) for r in clean ]

In [ ]:
print(sentence[1])

In [ ]:
print(text[1])

In [ ]:
def text_process(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc) 
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
tokens = x[0].split()
print(tokens)

In [ ]:
sample_text = "Hey there! This is a project"
print(text_process(sample_text))

In [ ]:
vector = CountVectorizer(analyzer=text_process).fit(x)
len(vector.vocabulary_)

In [ ]:
sample=x[20]
v = vector.transform([sample])
print(v)

In [ ]:
X = vector.transform(x)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=101)

In [ ]:
from sklearn.linear_model import LogisticRegression
LRmodel = LogisticRegression()
LRmodel.fit(X_train, y_train)

In [ ]:
pred= LRmodel.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm = confusion_matrix(y_test, pred)
print(cm)
print('\n')
print(classification_report(y_test, pred))
LRmodel.score(X_train, y_train)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, pred)
print(accuracy)

In [ ]:
from sklearn.metrics import recall_score
recall = recall_score(y_test, pred, average=None)
print("Recall :",recall)

In [ ]:
from sklearn.metrics import precision_score
precision = precision_score(y_test, pred, average=None) 
print("Precision :",precision)

In [ ]:
f1s = 2 * (precision * recall) / (precision + recall)
print("F-1 Score :",f1s)

In [ ]:
pred1 = Senti['Summary'][544]
pred1

In [ ]:
finaldata['Sentiment'][544]

In [ ]:
op = vector.transform([pred1])
LRmodel.predict(op)[0]